In [1]:
# allows to import of modules
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
nb_dir
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
## BADD-- supressing f1 score ill-defined and being set to 0.0 in labels with no true samples warning
import warnings
warnings.filterwarnings("ignore")

In [4]:
from pathlib import Path
import pandas as pd
import numpy as np

In [5]:
import  datatset
from torch.utils.data import DataLoader

In [6]:
# DataLoader

In [7]:
! ls ~/data-1/nybg2020/train/models

model-10993.pth  model-11596.pth


In [8]:
# TRAIN_PATH = Path("/home/shaun/personal/kaggle-data/nybg2020/train/")
TRAIN_PATH = Path("/home/ubuntu/data-1/nybg2020/train/")
df = pd.read_csv(TRAIN_PATH/"train.csv")
df_train = df[[ "file_name", "category_id"]]

In [9]:
def over_sample_class(x):
    if len(x) < 20:
        return x.sample(n=20, replace=True) 
    return x

In [10]:
def sample_class(x):
    if len(x) > 100:
        return x

In [11]:
# Update class labels
classes = df_train["category_id"].unique()
classes_map = {str(classes[i]):i for i in range(len(classes))}
import json

# save mapping
with open(TRAIN_PATH/'classes-map.json', 'w') as fp:
    json.dump(classes_map, fp)

In [12]:
len(df_train), len(df_train["category_id"].value_counts())

(1030747, 32093)

In [13]:
# only selected labels with more the
counts  = df_train["category_id"].value_counts()
labels_keep = counts.keys()[counts > 50] #
new_df = df_train[df_train["category_id"].isin(labels_keep)]

In [14]:
# new_df = df_train.groupby('category_id').apply(sample_class).reset_index(drop = True)

In [15]:
len(new_df), len(new_df["category_id"].value_counts())

(718266, 4922)

In [16]:
# new_df = new_df.groupby('category_id').apply(lambda x: x.sample(n=50)).reset_index(drop = True)

In [17]:
# # apply data augmentation to labels with less then 21 exampels
# labels_aug = counts.keys()[counts < 21] # Get labels that have less then 9 exmaples
# augment = new_df["category_id"].isin(labels_aug) # Get list of items to apply aug

In [18]:
new_df["augment"] = np.zeros(len(new_df)) == 1 # sett all augemntaion to zero

In [19]:
np.sum(new_df.augment)

0

In [20]:
# label_counts  = df["category_id"].value_counts()

In [21]:
# Update class labels
new_labels = [classes_map[str(i)] for i in new_df["category_id"]]
new_df["new_labels"] = new_labels

In [22]:
num_calsses = len(classes)

In [23]:
num_calsses

32093

In [24]:
from tqdm import tqdm_notebook

# there is a better way to do this
def sample(df, n=4):   
    validation_set = {}
    bool_arr = []
    for i,row in tqdm_notebook(df.iterrows(), total=df.shape[0]):
        if validation_set.get(row["new_labels"], 0) == n:
            bool_arr.append(False) 
        else:
            validation_set[row["new_labels"]] = validation_set.get(row["new_labels"], 0) + 1
            bool_arr.append(True) 

        
    return bool_arr

In [25]:
validation = sample(new_df, n=3)

In [26]:
new_df["is_valid"] = validation

In [27]:
train_df, valid_df = new_df[~new_df["is_valid"]] , new_df[new_df["is_valid"]]

In [28]:
# len(train_df), len(valid_df)
# (703500, 14766)

In [29]:
len(train_df), len(valid_df)

(703500, 14766)

In [30]:
valid_df.to_csv(TRAIN_PATH/"validation.csv")

In [31]:
ds = datatset.HerbDataSet(train_df, TRAIN_PATH, 256, label="new_labels")

In [32]:
data_loader = DataLoader(ds, batch_size=64, shuffle=True)

In [33]:
# Model

In [34]:
import model
import torchvision.models as models

In [35]:
m = model.get_model(models.resnet50(pretrained=True), p1=0, p2=0, device="gpu") # no drop out, trying to over fit

In [36]:
# m.load('/home/ubuntu/data-1/nybg2020/train/models/model-11596.pth')

In [37]:
import torch.nn as nn
import torch

In [38]:
assert m.fc[4].out_features == num_calsses

In [39]:
# Tracking

In [40]:
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter(TRAIN_PATH/"logs/run-6")

In [41]:
import metrics
import train

In [42]:
acc =  metrics.Accuracy()
f1 = metrics.F1()
manager = metrics.MetricManager([acc, f1], writer)

In [43]:
# Train

In [44]:
criterion = nn.CrossEntropyLoss()

In [45]:
layer_lrs = [
    {'params': m.layer1.parameters(), 'lr': 1e-6},
    {'params': m.layer2.parameters(), 'lr': 1e-6},
    {'params': m.layer3.parameters(), 'lr': 1e-5},
    {'params': m.layer4.parameters(), 'lr': 1e-4},
    {'params': m.fc.parameters(), 'lr': 1e-3}
]

In [46]:
optimizer = torch.optim.Adam( layer_lrs, lr=1e-3) # only training final layers

In [47]:
model_path=TRAIN_PATH/"models-5/"

In [48]:
!ls $model_path

model-3615.pth	model-7230.pth	model-8897.pth


In [49]:
trainer = train.Training(manager, criterion, optimizer, data_loader, 5, m, model_path )

In [50]:
! export CUDA_LAUNCH_BLOCKING=1

In [ ]:
trainer.run()

Epoch 0/5



Epoch 1/5



Epoch 2/5
